In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import matplotlib.pyplot as plt
from rtdl_revisiting_models import FTTransformer

from helpers import ModelPerformanceLog

import sys
sys.path.insert(0, '/home/wdwatson2/projects/CAT-Transformer/model')
from testingModel import CATTransformer, MyFTTransformer, Combined_Dataset, train, test, EarlyStopping, count_parameters
import for_rtdl

device_in_use = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device_in_use)

cuda


In [2]:

#GET California

# df_train = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/california/train.csv')
# df_test = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/california/test.csv')
# df_val = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/california/validation.csv') #READ FROM RIGHT SPOT

# df_train = pd.read_csv(r'C:\Users\smbm2\projects\CAT-Transformer\datasets\california\train.csv')
# df_test = pd.read_csv(r'C:\Users\smbm2\projects\CAT-Transformer\datasets\california\test.csv')
# df_val = pd.read_csv(r'C:\Users\smbm2\projects\CAT-Transformer\datasets\california\validation.csv') #READ FROM RIGHT SPOT

df_train = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/california/train.csv')
df_test = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/california/test.csv')
df_val = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/california/validation.csv')

cont_columns = [ 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude']
target = ['MedInc']
cat_columns=[]

#CHECKING TO MAKE SURE YOUR LIST IS CORRECT (NO NEED TO TOUCH)
yourlist = cont_columns + target
yourlist.sort()
oglist = list(df_train.columns)
oglist.sort()

cat_features=()

assert(yourlist == oglist), "You may of spelled feature name wrong or you forgot to put on of them in the list"

target_classes = [max(len(df_train[target].value_counts()), len(df_val[target].value_counts()),len(df_test[target].value_counts()))]
print(target_classes)
# Create a StandardScaler and fit it to the cont features
scaler = StandardScaler()
scaler.fit(df_train[cont_columns])

# Transform the training, test, and validation datasets
df_train[cont_columns] = scaler.transform(df_train[cont_columns])
df_test[cont_columns] = scaler.transform(df_test[cont_columns])
df_val[cont_columns] = scaler.transform(df_val[cont_columns])

#Wrapping in Dataset
train_dataset = Combined_Dataset(df_train, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])
val_dataset = Combined_Dataset(df_val, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])
test_dataset = Combined_Dataset(df_test, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])

batch_size = 256

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


#CAT
cat_model = CATTransformer(n_cont=len(cont_columns),
                       cat_feat=cat_features,
                       targets_classes=target_classes,
                       get_attn=False,
                       num_layers=1,
                       regression_on=True).to(device_in_use)

optimizer = torch.optim.Adam(params=cat_model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

early_stopping = EarlyStopping(patience=10, verbose=True, mode='min') #for regression

train_losses = []
train_rmse_1 = [] 
test_losses = []
test_rmse_1 = [] 

epochs = 400

for t in range(epochs):
    train_loss, train_rmse = train(regression_on=True, 
                                  get_attn=False,
                                   dataloader=train_dataloader, 
                                   model=cat_model, 
                                   loss_function=loss_function, 
                                   optimizer=optimizer, 
                                   device_in_use=device_in_use)
    test_loss, test_rmse = test(regression_on=True, 
                                  get_attn=False,
                                   dataloader=train_dataloader, 
                                   model=cat_model, 
                                   loss_function=loss_function, 
                                   device_in_use=device_in_use)
    train_losses.append(train_loss)
    train_rmse_1.append(train_rmse)
    test_losses.append(test_loss)
    test_rmse_1.append(test_rmse)

    epoch_str = f"Epoch [{t+1:2}/{epochs}]"
    train_metrics = f"Train: Loss {(train_loss)}, RMSE {(train_rmse)}"
    test_metrics = f"Test: Loss {(test_loss)}, RMSE {(test_rmse)}"
    print(f"{epoch_str:15} | {train_metrics:65} | {test_metrics:65}")

    early_stopping(test_rmse)
    
    if early_stopping.early_stop:
        print("Early stopping")
        break

[9851]
Epoch [ 1/400]  | Train: Loss 5.193376796287403, RMSE 2.19560762664728              | Test: Loss 3.638174057006836, RMSE 1.9012684633857326            
Epoch [ 2/400]  | Train: Loss 3.614687338209989, RMSE 1.8949875036875408            | Test: Loss 3.560952299519589, RMSE 1.882282905411302             
Epoch [ 3/400]  | Train: Loss 2.9391942317025705, RMSE 1.7016084842514574           | Test: Loss 2.2080772362257304, RMSE 1.482116184736553            
Epoch [ 4/400]  | Train: Loss 1.905451396055389, RMSE 1.3748023509979248            | Test: Loss 1.6686916915993941, RMSE 1.2875199443415593           
Epoch [ 5/400]  | Train: Loss 1.4377571543057759, RMSE 1.193960754494918            | Test: Loss 1.5628408883747302, RMSE 1.24812314384862             
Epoch [ 6/400]  | Train: Loss 1.285210112730662, RMSE 1.130036076955628             | Test: Loss 1.2538657836746752, RMSE 1.116260484645241            
Epoch [ 7/400]  | Train: Loss 1.2080854438898856, RMSE 1.0962108509582387        

In [3]:
#CAT
cat_model = MyFTTransformer(n_cont=len(cont_columns),
                       cat_feat=cat_features,
                       targets_classes=target_classes,
                       get_attn=False,
                       num_layers=1,
                       regression_on=True).to(device_in_use)

optimizer = torch.optim.Adam(params=cat_model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

early_stopping = EarlyStopping(patience=10, verbose=True, mode='min') #for regression

train_losses = []
train_rmse_1 = [] 
test_losses = []
test_rmse_1 = [] 

epochs = 400

for t in range(epochs):
    train_loss, train_rmse = train(regression_on=True, 
                                  get_attn=False,
                                   dataloader=train_dataloader, 
                                   model=cat_model, 
                                   loss_function=loss_function, 
                                   optimizer=optimizer, 
                                   device_in_use=device_in_use)
    test_loss, test_rmse = test(regression_on=True, 
                                  get_attn=False,
                                   dataloader=train_dataloader, 
                                   model=cat_model, 
                                   loss_function=loss_function, 
                                   device_in_use=device_in_use)
    train_losses.append(train_loss)
    train_rmse_1.append(train_rmse)
    test_losses.append(test_loss)
    test_rmse_1.append(test_rmse)

    epoch_str = f"Epoch [{t+1:2}/{epochs}]"
    train_metrics = f"Train: Loss {(train_loss)}, RMSE {(train_rmse)}"
    test_metrics = f"Test: Loss {(test_loss)}, RMSE {(test_rmse)}"
    print(f"{epoch_str:15} | {train_metrics:65} | {test_metrics:65}")

    early_stopping(test_rmse)
    
    if early_stopping.early_stop:
        print("Early stopping")
        break

Epoch [ 1/400]  | Train: Loss 4.607941962125008, RMSE 2.1087919829184547            | Test: Loss 3.844277151843958, RMSE 1.9546232934583698            
Epoch [ 2/400]  | Train: Loss 3.7079148250713683, RMSE 1.9197469322304976           | Test: Loss 3.5835742741300347, RMSE 1.8889808048281753           
Epoch [ 3/400]  | Train: Loss 3.0384796962403415, RMSE 1.7310896045283268           | Test: Loss 1.9783846704583419, RMSE 1.4003181603916905           
Epoch [ 4/400]  | Train: Loss 1.8177328883555897, RMSE 1.34063132185685             | Test: Loss 1.7941072865536338, RMSE 1.3376275217323972           
Epoch [ 5/400]  | Train: Loss 1.3870009246625399, RMSE 1.173615430530749            | Test: Loss 1.383191466331482, RMSE 1.1740278026513886            
Epoch [ 6/400]  | Train: Loss 1.2759044850081729, RMSE 1.125741067685579            | Test: Loss 1.2857237385030378, RMSE 1.1303558328695464           
Epoch [ 7/400]  | Train: Loss 1.2290444781905727, RMSE 1.1024125051080136           | Te